In [1]:
!nvidia-smi

Mon Feb 12 01:59:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000001:00:00.0 Off |                    0 |
| N/A   54C    P0             257W / 400W |  63225MiB / 81920MiB |     29%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
from modeling_vision import MM_LLMs, MM_LLMs_Config
from transformers import AutoModelForCausalLM, CLIPProcessor, CLIPModel,AutoModel, AutoTokenizer, AutoProcessor,AutoConfig,CLIPConfig, LlamaConfig, WhisperConfig, WhisperModel, LlamaModel, LlamaTokenizer
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch
import numpy as np
from torch import nn
from streaming import LocalDataset
from typing import List

In [4]:
MM_LLMs.register_for_auto_class()
MM_LLMs_Config.register_for_auto_class()

In [5]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint('vision-tinyllama-siglip-large')
latest

'vision-tinyllama-siglip-large/checkpoint-5900'

In [6]:
model = MM_LLMs.from_pretrained(
    latest,flash_attention = True, dtype = torch.bfloat16, torch_dtype = torch.bfloat16
)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


In [7]:
_ = model.cuda()

In [8]:
image_processor = AutoProcessor.from_pretrained('google/siglip-large-patch16-384')
tokenizer = AutoTokenizer.from_pretrained(latest)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from PIL import Image

def prepare_dataset(messages, images: List[str] = None):
    if images is not None:
        images = [Image.open(f).convert('RGB') for f in images]
        image_output = image_processor(images=images, return_tensors='pt')['pixel_values']
    else:
        image_output = None
    
    prompt = tokenizer.apply_chat_template(messages, tokenize = False)
    outputs = tokenizer(
                    prompt,
                    return_tensors='pt',
                    return_overflowing_tokens=False,
                    return_length=False
    )

    outputs['images'] = image_output
    outputs['image_index'] = torch.tensor([0] * len(outputs['images']))
    outputs['image_starts'] = torch.tensor([tokenizer.convert_tokens_to_ids('<image>')] * len(outputs['images']))
    return outputs

In [10]:
# !wget https://cdn.beautifulnara.net/wp-content/uploads/2017/12/10201620/Persian-cat-breed.jpg
# !wget https://www.jocooks.com/wp-content/uploads/2023/09/nasi-goreng-1-23.jpg

In [11]:
test_image = 'translated-LLaVA-Instruct-150K/filtered-llava-images/000000033471.jpg'
test_image2 = 'Persian-cat-breed.jpg'
test_image3 = 'abang-gay.png'
test_image4 = 'nasi-goreng-1-23.jpg'
images = [
    test_image,
    test_image2,
    test_image3,
    test_image4
]

In [12]:
for img in images:
    messages = [
        {'role': 'user', 'content': '<image> </image> ini gambar apa'},
    ]
    outputs = prepare_dataset(messages, images = [img])
    outputs['images'] = outputs['images'].type(model.dtype)
    for k in outputs.keys():
        if outputs[k] is not None:
            outputs[k] = outputs[k].cuda()

    with torch.no_grad():
        model_inputs = model.prepare_inputs_for_generation(**outputs)
    r = model_inputs.pop('input_ids', None)

    generate_kwargs = dict(
        model_inputs,
        max_new_tokens=300,
        top_p=0.95,
        top_k=50,
        temperature=0.1,
        do_sample=True,
        num_beams=1,
    )

    r = model.llm.generate(**generate_kwargs)
    print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu menggambarkan bas pelancongan putih yang memandu di jalan bandar yang sibuk. Terdapat beberapa orang di tempat kejadian, dengan seorang berdiri berhampiran tepi jalan dan seorang lagi lebih dekat dengan bas. Beberapa kereta juga boleh dilihat di jalan, termasuk satu di belakang bas dan satu lagi di sebelah kanan.

Selain itu, terdapat beberapa lampu isyarat yang kelihatan di sepanjang jalan, dengan satu di sebelah kiri dan satu lagi di sebelah kanan. Kehadiran lampu isyarat menunjukkan bahawa kawasan itu mungkin mempunyai sistem pengurusan trafik yang teratur.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu adalah gambar hitam putih seekor kucing putih yang cantik dengan mata oren, duduk di atas perabot hitam.</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Imej itu menunjukkan seorang lelaki muda berdiri di dalam bilik dengan tingkap, memakai baju hitam dan baju merah. Dia tersenyum dan kelihatan berada di dalam bilik yang gelap.</s>
<s>Imej itu memaparkan mangkuk putih yang diisi dengan nasi goreng yang dihiasi dengan lobak merah. Nasi goreng kelihatan rangup dan dihidangkan dengan lobak merah yang dihiris, menjadikannya hidangan yang menarik dan sihat.</s>


In [16]:
messages = [
    {'role': 'user', 'content': '<image> </image> <image> </image> apa kaitan 2 gambar ni'},
]
outputs = prepare_dataset(messages, images = [test_image2, test_image])
ok = outputs
ok['labels'] = ok['input_ids']

for k in ok.keys():
    if ok[k] is not None:
        ok[k] = ok[k].cuda()
        
for k in ['images']:
    if ok[k] is not None:
        ok[k] = ok[k].type(model.dtype)
        
with torch.no_grad():
    model_inputs = model.prepare_inputs_for_generation(**ok)
r = model_inputs.pop('input_ids', None)
label = model_inputs.pop('labels', None)
label = label.detach().cpu().numpy()
ok['input_ids'].shape, model_inputs['inputs_embeds'].shape

generate_kwargs = dict(
    model_inputs,
    max_new_tokens=300,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)

r = model.llm.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>Tiada hubungan yang jelas antara gambar 1 dan gambar 2. Gambar 1 ialah gambar putih kucing baka kuda, manakala gambar 2 ialah gambar bas bandar di jalan. Satu-satunya sambungan yang mungkin ialah kedua-dua gambar mengandungi unsur-unsur kehidupan seharian: orang dalam gambar 1 dan tempat letak kereta bas bandar dalam gambar 2. Walau bagaimanapun, tiada hubungan khusus antara kedua-dua imej selain itu.</s>


In [17]:
model.push_to_hub('malaysian-tinyllama-1.1b-siglip-large-384-vision', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-large-384-vision/commit/00e21131e9e67c9304789e18dba0c3a0d5b5e601', commit_message='Upload MM_LLMs', commit_description='', oid='00e21131e9e67c9304789e18dba0c3a0d5b5e601', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
image_processor.push_to_hub('malaysian-tinyllama-1.1b-siglip-large-384-vision', organization='mesolitica', safe_serialization=True)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/utils/hub.py:667: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-large-384-vision/commit/50e66426db8e263590229b599c3c3c661327269e', commit_message='Upload processor', commit_description='', oid='50e66426db8e263590229b599c3c3c661327269e', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
tokenizer.push_to_hub('malaysian-tinyllama-1.1b-siglip-large-384-vision', organization='mesolitica', safe_serialization=True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-tinyllama-1.1b-siglip-large-384-vision/commit/55872690f1afafc23e472a715ebf3d50d608969b', commit_message='Upload tokenizer', commit_description='', oid='55872690f1afafc23e472a715ebf3d50d608969b', pr_url=None, pr_revision=None, pr_num=None)